# DiT Image Generation Test Notebook

Use this notebook to quickly verify recent model changes (e.g., newly added residual connections) by sampling images from a DiT checkpoint.

**Note:** This notebook works in both Google Colab and local Jupyter environments.



In [ ]:
import os
import sys

# Detect environment and setup paths
in_colab = 'google.colab' in sys.modules
print(f"Running in: {'Google Colab' if in_colab else 'Local environment'}")

if in_colab:
    # Clone the repo if in Colab
    if not os.path.exists('/content/DiT'):
        print("Cloning DiT repository...")
        import subprocess
        subprocess.run(['git', 'clone', 'https://github.com/facebookresearch/DiT.git'], cwd='/content')
    
    # Change to DiT directory
    os.chdir('/content/DiT')
    
    # Install dependencies
    print("Installing dependencies...")
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'diffusers', 'timm'])
    
    repo_root = '/content/DiT'
else:
    # Local environment - assume we're in the DiT directory
    repo_root = os.getcwd()
    # If models.py is not in current dir, try parent
    if not os.path.exists(os.path.join(repo_root, 'models.py')):
        repo_root = os.path.dirname(repo_root)

# Add to Python path
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

print(f"Repository root: {repo_root}")
print(f"Current working directory: {os.getcwd()}")

# Verify the path is correct
if os.path.exists(os.path.join(repo_root, 'models.py')):
    print("✓ Repository root found successfully")
else:
    print(f"\n⚠️  WARNING: models.py not found at {repo_root}")

import torch
from torchvision.utils import save_image

from diffusion import create_diffusion
from diffusers.models import AutoencoderKL
from download import find_model
from models import DiT_models
from PIL import Image
from IPython.display import display

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.set_grad_enabled(False)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Repository root: C:\Users\schoo\Documents\GitHub\DiT
Current working directory: /content
Python path: ['C:\\Users\\schoo\\Documents\\GitHub\\DiT', '/content', '/env/python']

⚠️  WARNING: models.py not found at C:\Users\schoo\Documents\GitHub\DiT
Please edit the repo_root variable in the cell above to point to your DiT directory


ModuleNotFoundError: No module named 'diffusion'

In [ ]:
# --- Configuration ---------------------------------------------------------
model_key = "DiT-XL/2"      # Choose from models.DiT_models
image_size = 256             # 256 or 512 for auto-downloaded checkpoints
vae_variant = "mse"          # "mse" or "ema" (Stable Diffusion VAE variants)
num_sampling_steps = 100     # More steps -> better quality, slower
cfg_scale = 4.0
class_labels = [207, 360, 387, 974, 88, 979, 417, 279]  # ImageNet class IDs
samples_per_row = 4
seed = 0
custom_ckpt = None          # Optional path to a custom DiT checkpoint
# ---------------------------------------------------------------------------

torch.manual_seed(seed)
latent_size = image_size // 8

if custom_ckpt is None:
    if model_key != "DiT-XL/2":
        raise ValueError("Automatic checkpoints are only available for DiT-XL/2. Provide custom_ckpt instead.")
    ckpt_path = f"DiT-XL-2-{image_size}x{image_size}.pt"
else:
    ckpt_path = custom_ckpt

print(f"Loading {model_key} from {ckpt_path}")
model = DiT_models[model_key](input_size=latent_size).to(device)
state_dict = find_model(ckpt_path)
model.load_state_dict(state_dict)
model.eval()

diffusion = create_diffusion(str(num_sampling_steps))
vae = AutoencoderKL.from_pretrained(f"stabilityai/sd-vae-ft-{vae_variant}").to(device)


In [ ]:
n = len(class_labels)
z = torch.randn(n, 4, latent_size, latent_size, device=device)
y = torch.tensor(class_labels, device=device)

# Classifier-free guidance setup:
z = torch.cat([z, z], dim=0)
y_null = torch.tensor([1000] * n, device=device)
y = torch.cat([y, y_null], dim=0)
model_kwargs = dict(y=y, cfg_scale=cfg_scale)

samples = diffusion.p_sample_loop(
    model.forward_with_cfg,
    z.shape,
    z,
    clip_denoised=False,
    model_kwargs=model_kwargs,
    progress=True,
    device=device,
)
samples, _ = samples.chunk(2, dim=0)
samples = vae.decode(samples / 0.18215).sample

output_path = "dit_test_samples.png"
save_image(samples, output_path, nrow=samples_per_row, normalize=True, value_range=(-1, 1))
print(f"Saved grid to {output_path}")
display(Image.open(output_path))
